# Redes Neurais

## Introdução

Vimos anteriormente qual o tipo de resposta esperamos com uma regressão. Veremos agora uma outra forma de prever valores numéricos porém utilizando redes neurais.

Vamos rapidamente discutir como é formada a estrutura de uma rede e como sua arquitetura se comporta.

## Exemplo de implementação de Rede Neural

![SVM01](https://cdn-images-1.medium.com/max/1600/1*DW0Ccmj1hZ0OvSXi7Kz5MQ.jpeg)

Cada nó desta rede executa a soma de cada peso recebido por seus nós anteriores e tem sua multiplicação propagada por uma função de ativação:

![SVM02](https://www.analyticsvidhya.com/wp-content/uploads/2016/03/2.-ann-structure.jpg)

A estrutura tenta representar o que ocorre em um neurônio quando estimulado:

![SVM03](https://www.codeproject.com/KB/AI/1205732/neuron.png)

![SVM04](https://cdn-images-1.medium.com/max/2000/1*1Jr-Lt9vcEOW2opvZyLbdA.png)

Diferente dos modelos apresentados anteriormente, uma rede neural trabalha melhor com um número maior de features e pode precisar de um número muito maior de entradas de treino para convergir de maneira satisfatória.

In [ ]:
from IPython.lib.display import YouTubeVideo

In [ ]:
YouTubeVideo('rEDzUT3ymw4', width=720, height=560)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from sklearn.datasets.samples_generator import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Gerando uma base de dados

`make_regression` cria uma amostra randômica para estudos que envolvem regressão.

---

| Parâmetro | Descrição |
|--|--|
| n_samples | O número de amostras |
| n_features | O número de features |
| noise | Desvio padrão do ruído aplicado |


In [ ]:
?make_regression

In [ ]:
X, y = make_regression(n_samples = 1000, n_features = 20, noise = 0.1)

In [ ]:
X[0]

In [ ]:
y[0]

## Separando nossa informação em treino / teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

## Padronizando os dados

In [ ]:
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train[:, None])
y_test = y_scaler.transform(y_test[:, None])

In [ ]:
X_train[0]

In [ ]:
y_train[0]

In [ ]:
plt.scatter(X_train[:, 12], y_train)

## Tensorflow

O [TensorFlow™](https://www.tensorflow.org/?hl=pt-br) é uma biblioteca de software de código aberto para computação numérica que usa gráficos de fluxo de dados.

[TensorFlow Playground](playground.tensorflow.org)

### Sequential()

`Sequential` é a classe que encapsula a lista de camadas que que dará forma ao modelo.

```python
model = Sequential()
model.add(Dense(32, input_shape = (500,)))
model.add(Dense(32))
```


### Dense

`Dense` é a classe que implementa uma camada do modelo.

---

| Parâmetro | Descrição |
|--|--|
| units | Dimensão da camada |
| activation | A função de ativação dos nós |
| input_shape | Dimensão dos inputs da rede |

### RMSPropOptimizer

`RMSPropOptimizer` classe que controla o otimizador da rede.

### keras.Sequential.compile

Compila o modelo, geralmente recebe como parâmetro `loss`, `optimizer`, `metrics`.

---

| Parâmetro | Descrição |
|--|--|
| loss | Função considerada para cálculo do erro de saída do modelo |
| optimizer | Função utilizada para otimizar o modelo |
| metrics | Lista de métricas utilizadas para medir o desempenho do modelo durante o treino e teste |

In [ ]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(X_train.shape[1],)))
    model.add(keras.layers.Dense(64, activation=tf.nn.relu))
    model.add(keras.layers.Dense(1))

    optimizer = tf.train.RMSPropOptimizer(0.001)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

model = build_model()
model.summary()

| Layer (type)  | Output Shape | Param # |
|--|--|--|
| dense_3 (Dense) | (None, 64) | 1344 |
| dense_4 (Dense) | (None, 64) | 4160 |
| dense_5 (Dense) | (None, 1) | 65 |

---

- Layer (type) - Retorna nome da camada e tipo
- Output Shape - Retorna dimensão do vetor
- Param # - Retorna a o resultado de $inputs \times nós + bias $

Ex. $20 \times 64 + 64 = 1344$

In [ ]:
history = model.fit(X_train, y_train, epochs = 500, verbose = True, validation_split = 0.2)

Podemos plotar o erro para nosso dado de treino e teste num gráfico.

Facilitando a verificação de overfit e underfit.

In [ ]:
def plot_history(history):
  plt.figure(figsize = (15, 10))
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error')
  plt.plot(history.epoch, np.array(history.history['mean_absolute_error']), label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']), label = 'Val loss')
  plt.legend()
  
plot_history(history)  

Vemos que houve pouco melhora em nosso modelo após a época 200. Podemos criar um gatilho de earlystopping.

Este gatilho irá automaticamente parar o treinamento caso ele não veja melhoria no modelo em um número definido de épocas.

Para isso, devemos modificar o nosso modelo:

In [ ]:
model = build_model()

early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 20)

history = model.fit(X_train, y_train, epochs = 500, validation_split = 0.2, verbose = 0, callbacks = [early_stop])

plot_history(history)

Calculando o erro obtido no teste do modelo

In [ ]:
[loss, mae] = model.evaluate(X_test, y_test)

print("Mean Abs Error: {:.4f}".format(mae))

Vamos ver como o modelo se comporta ao prever pontos.

In [ ]:
test_preds = model.predict(X_test).flatten()

plt.figure(figsize = (15, 10))
plt.scatter(y_test, test_preds )
plt.xlabel('Valores reais')
plt.ylabel('Previsões')
plt.plot([5, -5], [5, -5])

## Como identificar Overfitting e Underfitting?

Overfiting e Underfitting são o caso de resultados ruins em modelos de inteligência artificial.

Podemos classificar como:

- **Underfitting**
 - O modelo tem resultados ruins com a parcela de treino e validação, ou seja, o modelo não é capaz de entregar o resultado esperado por causa de sua arquitetura ou da qualidade da informação que lhe é apresentada.
 

- **Overfitting**
 - O modelo tem resultados excelentes com sua parcela de treinamento, mas é incapaz de reproduzir os mesmos resultados com a parcela de validação.
 
 ![PIC2](https://shapeofdata.files.wordpress.com/2013/02/overfitting.png)

#### Underfitting

- Quando verificamos que o erro da parcela de treino é menor que a de validação e existe uma tendência de queda na parcela de validação, ou seja, esse valor de erro pode melhorar com um número maior de épocas.

 ![PIC3](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/07/Diagnostic-Line-Plot-Showing-an-Underfit-Model.png)

- Outro exemplo é quando temos um comportamento parecido entre a curva de treino e validação, porém, temos uma variação no erro entre estas curvas. Esse comportamento pode ser causado por um *underfit* da rede e, neste caso, pode ser corrigido modificando a estrutura da rede.


 ![PIC4](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/07/Diagnostic-Line-Plot-Showing-an-Underfit-Model-via-Status.png)

In [ ]:
YouTubeVideo('0h8lAm5Ki5g', width = 720, height = 560)

#### Overfitting

- Quando verificamos que o erro da parcela de treino melhora quanto maior é o número de épocas, porém, a parcela de validação tem um comportamento completamente oposto ou vemos este erro diminuir até um determinado ponto e depois degradar.

 ![PIC3](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/07/Diagnostic-Line-Plot-Showing-an-Overfit-Model.png)

Podemos combater o *overfit* da rede adicionando uma regularização dos peso do modelo ou uma política de dropout.

#### Regularização de peso

- **L1 regularization**
 - O custo adicionado é proporcional a soma dos valores absolutos dos pesos.

- **L2 regularization**
 - O custo adicionado é proporcional a soma dos valores quadráticos dos pesos.
 
Em poucas palavras, L1 tem o mesmo efeito de reduzir o número de inputs da rede, fazendo com que inputs que tem peso pequeno se aproximem de zero e reduzindo o ruído causado. L2 resulta valores de peso geral menores e estabiliza os pesos quando há alta correlação entre os recursos de entrada.

#### Dropout

Dropout se resume em, aleatoriamente, desligar conexões entre os nós de duas camadas. Diminuindo assim a complexidade da arquitetura da rede.

 ![PIC3](https://cdn-images-1.medium.com/max/800/1*iWQzxhVlvadk6VAJjsgXgg.png)



In [ ]:
YouTubeVideo('DEMmkFC6IGM', width = 720, height = 560)

## Resumo

- O erro quadrático médio (MSE) é comumente utilizado na predição de valores numéricos discretos em problemas de regressão linear;
- Diferente de problemas de classificação; Para problemas de regressão é comum utilizar métricas como o erro médio absoluto (MAE);
- Quando nossos inputs estão em _ranges_ distintos, é uma boa prática quando falamos de modelos que dependem de cálculo de distâncias a padronização ou normalização destes valores;
- Se o número de amostras for pequeno, de preferência sempre a redes com poucas camadas escondidas para evitar _overfit_ da rede.
- EarlyStopping é um bom meio de evitar o _overfit_.

# Classificação com NN

Criar um modelo de rede neural para resolver o problema da aula passada. Utilize os mesmos meios demonstrados acima. Para facilitar, vou importar os dados em um formato de dataset.

In [ ]:
dados = pd.read_csv('https://github.com/pgiaeinstein/comp_cog/raw/master/custo_colaborador_filtro.csv')
dados.head()

In [ ]:
X = dados.iloc[:, :6].values
y = dados.iloc[:, -1].values

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [ ]:
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train[:, None])
y_test = y_scaler.transform(y_test[:, None])

In [ ]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(12, activation=tf.nn.relu, input_shape=(X_train.shape[1],)))
    model.add(keras.layers.Dense(12, activation=tf.nn.relu))
    model.add(keras.layers.Dense(1))

    optimizer = tf.train.RMSPropOptimizer(0.001)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

model = build_model()
model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 20)

history = model.fit(X_train, y_train, epochs = 500, validation_split = 0.2, verbose = 0, callbacks = [early_stop])

plot_history(history)

In [ ]:
[loss, mae] = model.evaluate(X_test, y_test)

print("Mean Abs Error: {:.4f}".format(mae))

Com aquilo que aprendemos, vamos tentar resolver um problema de classificação binária. Veja como é simples apenas modificando um pouco a estrutura de nossa rede.

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples = 100000)

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .5)

In [ ]:
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
X_train[0]

Vamos criar nossa função `build_model` verifique que agora temos um problema de classificação binário, sendo assim, iremos utilizar como função de loss a `binary_crossentropy`.

In [ ]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(20, input_dim = 20, activation=tf.nn.relu))
    model.add(keras.layers.Dense(40, activation=tf.nn.relu))
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

Verifique que modificamos também nossa função de otimização e métrica. Para entender melhor como o algoritmo de otimização "adam" funciona, veja o seguinte [artigo](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/).

Vamos modificar nossa função para plotar os gráficos

In [ ]:
def plot_history(history):
  plt.figure(figsize = (15, 10))
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(history.epoch, np.array(history.history['acc']), label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_acc']), label = 'Val loss')
  plt.legend()

In [ ]:
model = build_model()

history = model.fit(X_train, y_train, epochs = 5, validation_split = 0.2, verbose = False)

plot_history(history)

In [ ]:
[loss, acc] = model.evaluate(X_test, y_test)

print("Acc: {:.4f}".format(acc))

In [ ]:
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc

Podemos plotar nossa matriz de confusão:

In [ ]:
test_preds = model.predict(X_test).flatten()
test_preds_bin = (test_preds > 0.5)
test_preds_bin

In [ ]:
cm  = confusion_matrix(y_test, test_preds_bin)

sns.heatmap(cm, annot = True, fmt='g', cmap = 'winter')

## Curva ROC

A Curva de operação do receptor (*Receiver Operationg Characteristic*)  é uma representação gráfica que ilustra o desempenho (ou performance) de um sistema classificador binário e como o seu limiar de discriminação é variado. [Wikipedia](https://pt.wikipedia.org/wiki/Caracter%C3%ADstica_de_Opera%C3%A7%C3%A3o_do_Receptor)

 ![PIC6](https://ncss-wpengine.netdna-ssl.com/wp-content/uploads/2016/06/ROC-Curves-Empirical-19.png)

#### Sensitivity

Também chamada de *true positive rate*, *recall* ou *probabilidade de detecção*. Mede a proporção de positivos realmente detectados como positivos.

$$
TPR = \frac{TP}{TP + FN}
$$

#### Specificity

Também chamada de *true negative rate*, mede a proporção de negativos realmente detectados como negativos.

$$
TNR = \frac{TN}{TN + FP}
$$

In [ ]:
YouTubeVideo('21Igj5Pr6u4', width = 720, height = 560)

Obtendo os valores:

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, test_preds)
auc_calc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize = (15, 10))
plt.plot(fpr, tpr, label='Modelo (area = {:.3f})'.format(auc_calc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('Curva ROC')
plt.legend(loc='best')
plt.plot([0, 1], [0, 1], 'k--')